In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
!curl -O https://storage.googleapis.com/tf-datasets/titanic/train.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30874  100 30874    0     0  14264      0  0:00:02  0:00:02 --:--:-- 14260


In [8]:
df = spark.read.option("header", "true").csv("train.csv")
df.show()

+--------+------+----+------------------+-----+-------+------+-------+-----------+-----+
|survived|   sex| age|n_siblings_spouses|parch|   fare| class|   deck|embark_town|alone|
+--------+------+----+------------------+-----+-------+------+-------+-----------+-----+
|       0|  male|22.0|                 1|    0|   7.25| Third|unknown|Southampton|    n|
|       1|female|38.0|                 1|    0|71.2833| First|      C|  Cherbourg|    n|
|       1|female|26.0|                 0|    0|  7.925| Third|unknown|Southampton|    y|
|       1|female|35.0|                 1|    0|   53.1| First|      C|Southampton|    n|
|       0|  male|28.0|                 0|    0| 8.4583| Third|unknown| Queenstown|    y|
|       0|  male| 2.0|                 3|    1| 21.075| Third|unknown|Southampton|    n|
|       1|female|27.0|                 0|    2|11.1333| Third|unknown|Southampton|    n|
|       1|female|14.0|                 1|    0|30.0708|Second|unknown|  Cherbourg|    n|
|       1|female| 4.0

In [9]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col

In [10]:
df.withColumn("age", col('age').cast("int"))

DataFrame[survived: string, sex: string, age: int, n_siblings_spouses: string, parch: string, fare: string, class: string, deck: string, embark_town: string, alone: string]

In [12]:
# Сколько строк в таблице ?
df.count()

627

In [61]:
# Сколько строк, где пол указан мужской
# df.select(F.count('sex == male'))
# df.agg(F.count(F.col('sex' == "male"))).show()
df.groupby(col("sex")).agg(F.count(col('age'))).show()

+------+----------+
|   sex|count(age)|
+------+----------+
|female|       217|
|  male|       410|
+------+----------+



In [80]:
# С помощью одной команды посчитать количество выживших мужчин и женщин
surv = df.filter(F.col("survived") == 1)
# surv.show()
surv.groupby(col("sex")).agg(F.count(col('age'))).show()

+------+----------+
|   sex|count(age)|
+------+----------+
|female|       169|
|  male|        74|
+------+----------+



In [74]:
# С помощью одной команды определить средний возраст пассажиров каждого из Pclass`ов
df.groupby(col("class")).agg(F.mean(col('age'))).show()

+------+------------------+
| class|          avg(age)|
+------+------------------+
| First|37.056603773584904|
|Second| 29.93566929133858|
| Third|26.055718475073313|
+------+------------------+

